In [1]:
import numpy as np
from matplotlib import pyplot as plt
from numpy import random as rnd
import tensorflow as tf
import scipy.signal as sig 
import gym
import numpy.random as rnd

In [6]:
bandits = [0.01,0.04,-0.02,-0.03]
num_bandits = len(bandits)

def pull_bandit(bandit):
    rr = rnd.randn(1)
    if rr > bandit:
        return 1
    else:
        return -1

In [7]:
tf.reset_default_graph()

reward_container = tf.placeholder(shape=[1],dtype=tf.float32)
action_container = tf.placeholder(shape=[1],dtype=tf.int32)

weights = tf.Variable(tf.ones([num_bandits]))
chosen_action = tf.argmax(weights,axis=0)

weight_in_charge = tf.slice(weights,action_container,[1])
loss = -tf.log(weight_in_charge)*reward_container
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(loss)

In [8]:
with tf.Session() as sess:
    num_episodes = 7000
    rewards_scoreboard = np.zeros([num_bandits])
    e = 0.1
    
    sess.run(tf.global_variables_initializer())
    
    i = 0
    while i<num_episodes:
        
        if rnd.rand(1)<e:
            action = rnd.randint(0,num_bandits,1)
        else:
            action = sess.run([chosen_action])
        reward = pull_bandit(bandits[action[0]])
        
        _,wic,weights_vec = sess.run([optimizer,weight_in_charge,weights],feed_dict={
            reward_container:[reward],
            action_container:[action[0]]
        })
        
        rewards_scoreboard[action]+=reward
        if i%50==0:
            print("Running reward for the " + str(num_bandits) + " bandits: " + str(rewards_scoreboard))
        i+=1

print("The agent thinks bandit " + str(np.argmax(weights_vec)) + " is the most promising....")
if np.argmax(weights_vec) == np.argmax(-np.array(bandits)):
    print("...and it was right!")
else:
    print("...and it was wrong!")

Running reward for the 4 bandits: [ 1.  0.  0.  0.]
Running reward for the 4 bandits: [ -2.   1.  -2.  20.]
Running reward for the 4 bandits: [ -1.   3.  -2.  11.]
Running reward for the 4 bandits: [ -1.   4.   0.  16.]
Running reward for the 4 bandits: [ -3.   4.   0.  14.]
Running reward for the 4 bandits: [ -3.   5.  -3.  16.]
Running reward for the 4 bandits: [ -4.   6.  -3.  26.]
Running reward for the 4 bandits: [ -4.   6.  -2.  27.]
Running reward for the 4 bandits: [ -3.   6.  -2.  26.]
Running reward for the 4 bandits: [ -4.   5.  -2.  22.]
Running reward for the 4 bandits: [ -4.   6.  -1.  14.]
Running reward for the 4 bandits: [ -4.   6.  -1.  16.]
Running reward for the 4 bandits: [ -4.   6.  -1.  22.]
Running reward for the 4 bandits: [ -6.   6.  -1.  32.]
Running reward for the 4 bandits: [ -6.   6.   0.  31.]
Running reward for the 4 bandits: [ -5.   6.   0.  26.]
Running reward for the 4 bandits: [ -5.   8.  -2.  20.]
Running reward for the 4 bandits: [ -5.   7.  -2.  2